In [ ]:
# !pip install torchsummary

In [1]:
import os, time, copy
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import torchvision

from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from torchvision.datasets.folder import ImageFolder
from torchvision.models import resnet18

import matplotlib.pyplot as plt
from torchsummary import summary
%matplotlib inline

# Pretrained model불러오기 (resnet18)

In [2]:
model = resnet18(weights=torchvision.models.ResNet18_Weights)

C:\Users\user\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
summary(model, input_size=(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

## 두가지 Case에 대해서 Transfer learning을 적용해볼겁니다.

<span style = 'font-size:1.3em;line-height:1.5em'><b>Case 1. </b>resnet의 모든 weight들을 전부 우리 dataset에 맞게 fine tuning</span>

<span style = 'font-size:1.3em;line-height:1.5em'><b>Case 2. </b>resnet의 마지막 부분인 Linear Classifier만 fine tuning. 이전의 모든 weight들은 전부 그대로 fix!</span>

In [5]:
import os
from download import download_dataset

# 요부분은 빼셔도 됩니다.
# 제 local PC에서 matplotlib 사용시 커널이 죽는 문제가 있어서 사용하는 부분입니다.
# 문제가 발생하지 않는 분들은 빼도 됩니다.
os.environ['KMP_DUPLICATE_LIB_OK']='True' 


# 개, 고양이 데이터셋 다운로드를 받습니다.
# 받은 데이터는 tmp/PetImages 폴더 안에 저장됩니다.
# 중간에 Truncated 된 이미지가 존재합니다.

# 데이터가 없으면 아래 코드를 주석 해제하고 실행하세요.
# download_dataset('https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip', 
#                  'PetImages')

In [6]:
data_transforms = transforms.Compose([
#     transforms.RandomHorizontalFlip(),
    transforms.Resize((224, 224)), 
    transforms.ToTensor()
])

my_dataset = ImageFolder(root='data/cats_and_dogs/PetImages/',
                         transform=data_transforms)

### 본격적으로 학습을 하기 전, 데이터를 train, validation으로 나누는 함수를 작성합시다.

- <span style = 'font-size:1.2em;line-height:1.5em'>Dataset에 어떤 클래스가 있는지 확인하기</span>

In [7]:
my_dataset.class_to_idx

{'Cat': 0, 'Dog': 1}

In [8]:
tgts = np.array(my_dataset.targets)
tgts

array([0, 0, 0, ..., 1, 1, 1])

- <span style = 'font-size:1.2em;line-height:1.5em'>Dataset을 입력받으면 각 클래스별로 train, validataion에 사용될 index를 반환하는 함수</span>

In [9]:
def get_trn_val_idxs(dset, trn_ratio=0.8):
    tgts = np.array(dset.targets)
    trn_idxs = []
    val_idxs = []
    for class_val in dset.class_to_idx.values():
        length = len(tgts)
        split_idx = int(np.floor(trn_ratio * length))
        
        idxs = np.argwhere(tgts == class_val).reshape(-1)
        np.random.shuffle(idxs)
        class_trn_idxs, class_val_idxs = idxs[:split_idx], idxs[split_idx:]
        
        trn_idxs.extend(class_trn_idxs)
        val_idxs.extend(class_val_idxs)
        
    trn_idxs = np.array(trn_idxs)
    val_idxs = np.array(val_idxs)
    return trn_idxs, val_idxs

In [10]:
trn_idxs, val_idxs = get_trn_val_idxs(my_dataset)

- <span style = 'font-size:1.2em;line-height:1.5em'>`SubsetRandomSampler`: 데이터에서 샘플링 대상이 될 index를 입력받으면, 그 인덱스에 해당하는 데이터에서 random 하게 batch_size 만큼 데이터를 샘플링</span>

In [12]:
trn_sampler = SubsetRandomSampler(trn_idxs)
val_sampler = SubsetRandomSampler(val_idxs)

- <span style = 'font-size:1.2em;line-height:1.5em'>`SubsetRandomSampler`를 `DataLoader`에 넣어서 Mini-batch 생성기를 정의</span>

In [13]:
# sampler를 사용하려면 shuffle은 반드시 False로 설정
trn_loader = DataLoader(dataset=my_dataset,
                        shuffle=False,
                        batch_size=16,
                        sampler=trn_sampler)

val_loader = DataLoader(dataset=my_dataset,
                        shuffle=False,
                        batch_size=16,
                        sampler=val_sampler)

In [14]:
for i, (x, y) in enumerate(trn_loader):
    if i == 0:
        break

<span style = 'font-size:1.5em;line-height:1.5em'><b>Case 1. </b>resnet의 모든 weight들을 전부 우리 dataset에 맞게 fine tuning</span>

### 이제 Pretrained 모델을 불러와서 우리 데이터에 맞게 사용합시다.

- <span style = 'font-size:1.2em;line-height:1.5em'>그전에 모델의 구조를 한번 볼까요?</span>

In [15]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

- <span style = 'font-size:1.2em;line-height:1.5em'>다른건 다 그대로 사용해도 하나는 꼭 변경해야 합니다.</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>마지막의 `(fc)`부분은 out_features=2가 되어야 합니다. (2개의 class밖에 없으므로)</span>

In [16]:
model.fc = nn.Linear(in_features=512, out_features=2, bias=True)

- <span style = 'font-size:1.2em;line-height:1.5em'>바뀐 모델의 구조를 봅시다.</span>

In [17]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### 이제 학습해봅시다. (지난주 실습 자료 중, 일부를 가져옵니다.)

- <span style = 'font-size:1.2em;line-height:1.5em'>`train()`함수는 각 iteration마다 다음과 같이 진행됩니다.</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 1.</b> batch_loader로부터 mini-batch x, y 데이터를 획득하고 원하는 device에 위치시키기</span>
        - <span style = 'font-size:1.0em;line-height:1.5em'> <b>n_data: </b>mini-batch data수, <b>1: </b>channel수(흑백이라서 단일 채널. 칼라 이미지(RGB)는 기본으로 3으로 설정됨) <b>28: </b>Width, <b>28: </b>height</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 2.</b> 지난 batch로부터 계산했던 gradient를 초기화(`zero_grad()`)</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 3.</b> 모델에 batch x를 입력하여 forward propagation</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 4.</b> loss function에 모델이 예측한 각 클래스에 속할 확률(`y_pred_prob`)과 실제 레이블 (`y`)을 넣어서 loss 계산</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 5.</b> Backpropagation으로 각 parameter의 gradient를 계산</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 6.</b> Gradient Descent로 parameter값 update</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 7.</b> `trn_loss` 변수에 mini-batch loss를 누적해서 합산</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 8.</b> 데이터 한 개당 평균 train loss 산출</span>

In [18]:
def train(model, data_loader, optimizer, criterion, device):
    model.train()
    trn_loss = 0
    # 매 mini-batch train data마다 반복
    for i, (x, y) in enumerate(data_loader):
        # 1-(1): 모델에 입력하기 위해서 데이터의 형태 변환
        x = x.to(device) # x.shape: (batch_size,1, 28,28)
        y = y.to(device)
        
        # 1-(2): 기존에 계산된 gradient를 0으로 reset
        my_opt.zero_grad()
        
        # 1-(3): Forward Propagation
        y_pred_prob = model(x)
        
        # 1-(4): Loss Calculation
        loss = criterion(y_pred_prob, y)
        
        # 1-(5): Gradient Calculation(Backprop)
        loss.backward()
        
        # 1-(6): Update parameter
        optimizer.step()
        
        # 1-(7): trn_loss에 mini_batch loss를 누적해서 계산하기
        trn_loss += loss.item()
    
    # Step8. 데이터 한 개당 평균 train loss
    avg_trn_loss = trn_loss / len(data_loader.dataset)
    return avg_trn_loss

- <span style = 'font-size:1.2em;line-height:1.5em'>`evaluate()`함수는 각 iteration마다 다음과 같이 진행됩니다.</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 1.</b> batch_loader로부터 mini-batch x, y 데이터를 획득하고 원하는 device에 위치시키기</span>
        - <span style = 'font-size:1.0em;line-height:1.5em'> <b>n_data: </b>mini-batch data수, <b>1: </b>channel수(흑백이라서 단일 채널. 칼라 이미지(RGB)는 기본으로 3으로 설정됨) <b>28: </b>Width, <b>28: </b>height</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 2.</b> 모델에 batch x를 입력하여 forward propagation</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 3.</b> loss function에 모델이 예측한 각 클래스에 속할 확률(`y_pred_prob`)과 실제 레이블 (`y`)을 넣어서 loss 계산</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 4.</b> 모델이 예측하는 레이블을 산출 (with `torch.argmax()`)</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 5.</b> Minibatch의 실제 레이블(`y`)과 예측 레이블(`y_pred_label`)을 누적하여 저장</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 6.</b> `eval_loss` 변수에 mini-batch loss를 누적해서 합산</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 7.</b> 데이터 한 개당 평균 evaluation loss와 accuracy 산출</span>

In [19]:
def evaluate(model, data_loader, optimizer, criterion, device):
    model.eval() # 모델을 평가모드로!
    eval_loss = 0
    
    results_pred = []
    results_real = []
    with torch.no_grad(): # evaluate()함수에는 단순 forward propagation만 할 뿐, gradient 계산 필요 X.
        for i, (x, y) in enumerate(data_loader):
            # Step 1. mini-batch에서 x,y 데이터를 얻고, 원하는 device에 위치시키기
            x = x.to(device) # x.shape: [batch_size,28,28] -> [batch_size, 784]
            y = y.to(device)

            # Step 2. Forward Propagation
            y_pred_prob = model(x)

            # Step 3. Loss Calculation
            loss = criterion(y_pred_prob, y)
            
            # Step 4. Predict label
            y_pred_label = torch.argmax(y_pred_prob, dim=1)
            
            # Step 5. Save real and predicte label
            results_pred.extend(y_pred_label.detach().cpu().numpy())
            results_real.extend(y.detach().cpu().numpy())
            
            # Step 6. eval_loss변수에 mini-batch loss를 누적해서 합산
            eval_loss += loss.item()

    # Step 7. 데이터 한 개당 평균 eval_loss와 accuracy구하기
    avg_eval_loss = eval_loss / len(data_loader.dataset)
    results_pred = np.array(results_pred)
    results_real = np.array(results_real)
    accuracy = np.sum(results_pred == results_real) / len(results_real)
    
    return avg_eval_loss, accuracy

- <span style = 'font-size:1.2em;line-height:1.5em'>매 Epoch에 드는 시간 측정</span>

In [20]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

- <span style = 'font-size:1.2em;line-height:1.5em'>Dataset과 Mini-batch를 자동으로 생성할 DataLoader준비하기</span>

In [21]:
# 위에서 my_dataset은 이미 정의 했음
# sampler를 사용하려면 shuffle은 반드시 False로 설정
trn_loader = DataLoader(dataset=my_dataset,
                        shuffle=False,
                        batch_size=16,
                        sampler=trn_sampler)

val_loader = DataLoader(dataset=my_dataset,
                        shuffle=False,
                        batch_size=16,
                        sampler=val_sampler)

- <span style = 'font-size:1.2em;line-height:1.5em'>연산을 수행할 device를 설정하기</span>

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

- <span style = 'font-size:1.2em;line-height:1.5em'>모델에 대한 객체 생성하기</span>

In [24]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

- <span style = 'font-size:1.2em;line-height:1.5em'>학습한 모델을 저장할 directory 생성하기</span>

In [25]:
save_dir = 'models'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

- <span style = 'font-size:1.2em;line-height:1.5em'>필요한 hyperparameter값 설정하기</span>

In [26]:
N_EPOCHS = 10
LR = 2e-4
BATCH_SIZE = 2**9

- <span style = 'font-size:1.2em;line-height:1.5em'>loss함수 정의하기</span>

In [27]:
loss_func = nn.CrossEntropyLoss(reduction='sum')

- <span style = 'font-size:1.2em;line-height:1.5em'>optimizer 생성하기</span>

In [28]:
my_opt = optim.Adam(model.parameters(), lr = LR)

- <span style = 'font-size:1.2em;line-height:1.5em'>trn_data에 대해서 train()함수를, tst_data에 대해서 evaluate()함수를 반복적으로 호출하면서 모델을 학습</span>
    - <span style = 'font-size:1.2em;line-height:1.5em'>매 epoch마다 학습이 마무리되면, 모델 평가를 진행한다</span>

In [29]:
best_val_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    trn_loss = train(model=model, 
                     data_loader=trn_loader, 
                     optimizer=my_opt, 
                     criterion=loss_func,
                     device=device)
    val_loss, accuracy = evaluate(model=model, 
                                  data_loader=val_loader, 
                                  optimizer=my_opt, 
                                  criterion=loss_func,
                                  device=device)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), f'{save_dir}/my_model2.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {trn_loss:.3f} | Test Loss: {val_loss:.3f} | Test Acc: {100*accuracy:.3f}% ')

KeyboardInterrupt: 

In [ ]:
my_dataset = ImageFolder(root='data/cats_and_dogs/PetImages/', transforms=data_transforms)
original_loader

In [ ]:
# 이미지 폴더로부터 데이터를 로드합니다.
original_dataset = ImageFolder(root='tmp/PetImages',                            # 이미지가 들어있는 root 경로를 지정합니다.
                               transform=transforms.Compose([                   # Resize 후 정규화(0~1)를 수행합니다.
#                                    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)), # 개와 고양이 사진 파일의 크기가 다르므로, Resize로 맞춰줍니다.
                                   transforms.ToTensor()                        # PIL image data를 pytorch tensor로 변환합니다.
                               ]))

# DataLoader를 생성한다.
original_loader = DataLoader(original_dataset, # 이전에 생성한 original_dataset를 로드 합니다.
                             batch_size=1,     # 배치사이즈
                             shuffle=False,   # 셔플 여부
                             num_workers=1)

In [ ]:
torch.utils.data.random_split()

# Dataset 정의하기

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'data/data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=4,
                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

# 일부 이미지 시각화하기

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # 갱신이 될 때까지 잠시 기다립니다.


# 학습 데이터의 배치를 얻습니다.
inputs, classes = next(iter(dataloaders['train']))

# 배치로부터 격자 형태의 이미지를 만듭니다.
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

## 학습하는 함수 생성하기

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # 각 에폭(epoch)은 학습 단계와 검증 단계를 갖습니다.
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # 모델을 학습 모드로 설정
            else:
                model.eval()   # 모델을 평가 모드로 설정

            running_loss = 0.0
            running_corrects = 0

            # 데이터를 반복
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # 매개변수 경사도를 0으로 설정
                optimizer.zero_grad()

                # 순전파
                # 학습 시에만 연산 기록을 추적
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # 학습 단계인 경우 역전파 + 최적화
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # 통계
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # 모델을 깊은 복사(deep copy)함
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # 가장 나은 모델 가중치를 불러옴
    model.load_state_dict(best_model_wts)
    return model

## 모델 예측값을 시각화하는 함수 생성하기

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'predicted: {class_names[preds[j]]}')
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

## 실제 학습 진행하기

In [ ]:
model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# 새로 생성된 모듈의 매개변수는 기본값이 requires_grad=True 임
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# 이전과는 다르게 마지막 계층의 매개변수들만 최적화되는지 관찰
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# 7 에폭마다 0.1씩 학습률 감소
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [ ]:
# model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
#                        num_epochs=25)
model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=25)
